In [ ]:
import os
import matplotlib.pyplot as plt
import pydicom
import numpy as np
from glob import glob
import pandas as pd
from random import shuffle
import cv2
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers. normalization import BatchNormalization
from sklearn.model_selection import train_test_split
from keras.models import model_from_json

In [ ]:
label = []
train_data = []
i = 0
Y = pd.read_csv('SPECT_4_04_2020.csv', usecols=['Subject', 'Group'])#reading 2 columns of CSV
for f in range(1579):
    path = os.path.join(r'E:\SPECT_Final',str(Y['Subject'][f]),r'Reconstructed_DaTSCAN\*\*\*.dcm')
    dataset = pydicom.dcmread(glob(path)[0]) #reading DCM image
    image = np.array(dataset.pixel_array) #converting into numpy array
    image1 = np.zeros(shape = (image.shape))
    #normalisation and binarization of the image and taking 40th slice only
    for j in range(image.shape[1]):
        for k in range(image.shape[2]):
            image1[40,j,k] = np.divide(image[40,j,k],np.max(image[40]))
            if(image1[40,j,k] > 0.70):
                image1[40,j,k] = 1
            else:
                image1[40,j,k] = 0
    #one-hoy encoding
    if(Y['Group'][f] == 'PD'):
        label = (np.array([1, 0]))
    else:
        label = (np.array([0, 1]))
    train_data.append([np.array(image1[40]), label]) #making train_data

In [ ]:
shuffle(train_data)

In [ ]:
trainImages = np.array([i[0] for i in train_data]).reshape(-1, 109, 91, 1)
trainLabels = np.array([i[1] for i in train_data])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(trainImages, trainLabels, test_size=0.20)

In [ ]:
model = Sequential()
model.add(Conv2D(32, kernel_size = (3, 3), activation='relu', input_shape=(109, 91, 1)))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(64, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Conv2D(96, kernel_size=(3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(2, activation = 'softmax'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
model.fit(X_train, y_train, batch_size = 8, epochs = 10, verbose = 1)

In [ ]:
loss, acc = model.evaluate(X_test, y_test, verbose = 0)